In [1]:
using SparseArrays
using LinearAlgebra, Arpack
using Plots, Statistics
using Random


In [8]:
# methods for sparse matrix
const σx = sparse([0.0 1.0; 1.0 0.0])
const σy = sparse([0.0 -im; im 0.0])
const σz = sparse([1.0 0.0; 0.0 -1.0])

A ⊗ B = kron(A, B)
kron(A, B, C...) = kron(A, kron(B, C...))

H = σx ⊗ σz
H = Hermitian(H)
eigs(H)


LoadError: MethodError: no method matching kron(::SparseMatrixCSC{Float64, Int64})
The function `kron` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  kron(::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[8]:7[24m[39m


In [2]:
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]
A ⊗ B = kron(A, B)

H = σx ⊗ σz ⊗ σy ⊗ σx + σy ⊗ σy ⊗ σz ⊗ σz
H = Hermitian(H)
@show ishermitian(H)
@show eigen(H)


ishermitian(H) = true
eigen(H) = Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}([-1.999999999999997, -1.999999999999996, -1.9999999999999956, -1.9999999999999938, -4.0167948125724077e-17, -1.812859280056657e-17, 2.320862976945754e-18, 1.2650663898016148e-17, 4.440892098500626e-16, 1.3322676295501878e-15, 1.7763568394002505e-15, 2.220446049250313e-15, 1.9999999999999998, 1.9999999999999998, 2.0, 2.0], ComplexF64[0.0 + 0.0im 0.0 + 0.0im 0.35355339059327373 - 0.35355339059327384im -5.551115123125783e-17 - 7.632783294297951e-17im 0.1593018323354129 + 0.17728877712639257im 0.16911113638144437 + 0.057635389378134484im -0.1289785479956944 + 0.14872285552189446im -0.48417052445485764 + 0.04266784043415899im 3.548815350498579e-17 - 0.3691552602994369im -3.3306690738754696e-16 + 3.8594145019595916e-16im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 2.7755575615628914e-17 + 0.0im -0.35355339059327384 - 0.35355339059327373im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.486126850835581

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
16-element Vector{Float64}:
 -1.999999999999997
 -1.999999999999996
 -1.9999999999999956
 -1.9999999999999938
 -4.0167948125724077e-17
 -1.812859280056657e-17
  2.320862976945754e-18
  1.2650663898016148e-17
  4.440892098500626e-16
  1.3322676295501878e-15
  1.7763568394002505e-15
  2.220446049250313e-15
  1.9999999999999998
  1.9999999999999998
  2.0
  2.0
vectors:
16×16 Matrix{ComplexF64}:
 0.0+0.0im           0.0+0.0im          …   0.0+0.0im
 0.0+0.0im           0.0+0.0im              0.0+0.0im
 0.0+0.0im          -0.5+0.0im              0.0+0.0im
 0.5+0.0im  -5.55112e-17+0.0im             -0.5+0.0im
 0.0-0.5im           0.0+5.55112e-17im      0.0+0.5im
 0.0+0.0im           0.0-0.5im          …   0.0+0.0im
 0.0+0.0im           0.0+0.0im              0.0+0.0im
 0.0+0.0im           0.0+0.0im              0.0+0.0im
 0.0+0.5im           0.0+0.0im              0.0+0.5im
 0.0+0.0im           0.0-0.5im              0.0

In [ ]:
repeat(σx, 4)
repeat(I, 2)
@show arrays = repeat(I, inner=(1, 2), outer=(1, 1))
@show arrays = repeat(I, inner=(1, 2), outer=(2, 1))
I ⊗ I


arrays = repeat(I, inner = (1, 2), outer = (1, 1)) = [1.0 1.0 0.0 0.0; 0.0 0.0 1.0 1.0]
arrays = repeat(I, inner = (1, 2), outer = (2, 1)) = [1.0 1.0 0.0 0.0; 0.0 0.0 1.0 1.0; 1.0 1.0 0.0 0.0; 0.0 0.0 1.0 1.0]


4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [ ]:
const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(ComplexF64, 2^N, 2^N)
    for i in 1:N
        j = i % N + 1
        H += model.J * (I ⊗ repeat(σz, inner=(1, i - 1), outer=(1, N - i))) ⊗ (I ⊗ repeat(σz, inner=(1, j - 1), outer=(1, N - j)))
        H += model.h * (I ⊗ repeat(σx, inner=(1, i - 1), outer=(1, N - i)))
    end
    return Hermitian(H)
end
model = TFIM()
model(2)


LoadError: DimensionMismatch: a has size (4, 4), b has size (16, 0), mismatch at dim 1

In [ ]:
const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(ComplexF64, 2^N, 2^N)
    ising = spzeros(ComplexF64, 2^N, 2^N)
    field = spzeros(ComplexF64, 2^N, 2^N)
    for i in 2:N
        j = i % N + 1
        ising = ising ⊗ I
        ising += 
    end
    return Hermitian(H)
end
model = TFIM()
model(2)


In [6]:
# methods for sparse matrix
const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

A ⊗ B = kron(A, B)
H = σx ⊗ σz
H = Hermitian(H)
eigen(H)


Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
4-element Vector{Float64}:
 -0.9999999999999989
 -0.9999999999999989
  1.0
  1.0
vectors:
4×4 Matrix{Float64}:
  0.707107  0.0       0.707107   0.0
  0.0       0.707107  0.0       -0.707107
 -0.707107  0.0       0.707107   0.0
  0.0       0.707107  0.0        0.707107

In [10]:
matarray = [I, I, I]
reduce(⊗, matarray)

const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1
        matising = replace(matarray_ising, i => σz, j => σz)
        matfield = replace(matarray_field, i => σx)
        H += model.J * reduce(⊗, matising)
        H += model.h * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
model = TFIM()
model(2)


4×4 Hermitian{Float64, Matrix{Float64}}:
 1.5  0.0  0.0  0.0
 0.0  1.5  0.0  0.0
 0.0  0.0  1.5  0.0
 0.0  0.0  0.0  1.5

In [ ]:
matarray = [I, I, I]
reduce(⊗, matarray)

const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1

        matising = copy(matarray_ising)
        matising[i] = σz
        matising[j] = σz
        matfield = copy(matarray_field)
        matfield[i] = σx
        H += model.J * reduce(⊗, matising)
        H += model.h * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
model = TFIM()
H = model(3)
(λ, v) = eigen(H)

@show λ


λ = [-2.1889010593167257, -2.188901059316723, -0.45685025174785654, -0.45685025174785565, 0.4568502517478572, 0.4568502517478594, 2.188901059316734, 2.188901059316734]


8-element Vector{Float64}:
 -2.1889010593167257
 -2.188901059316723
 -0.45685025174785654
 -0.45685025174785565
  0.4568502517478572
  0.4568502517478594
  2.188901059316734
  2.188901059316734

In [1]:
const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1

        matising = copy(matarray_ising)
        matising[i] = σz
        matising[j] = σz
        matfield = copy(matarray_field)
        matfield[i] = σx
        H += model.J * reduce(⊗, matising)
        H += model.h * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
@kwdef struct TFIML
    J::Float64 = 1.0
    h::Float64 = 1.0
    g::Float64 = 0.10
end
function (model::TFIML)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1

        matising = copy(matarray_ising)
        matising[i] = σz
        matising[j] = σz
        H += model.J * reduce(⊗, matising)

        matfield = copy(matarray_field)
        matfield[i] = σx
        H += model.h * reduce(⊗, matfield)
        matfield[i] = σz
        H += model.g * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
model = TFIML()
H = model(10)
(λ, v) = eigen(H)
@show λ


LoadError: UndefVarError: `spzeros` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
const I = [1.0 0.0; 0.0 1.0]
const σx = [0.0 1.0; 1.0 0.0]
const σy = [0.0 -im; im 0.0]
const σz = [1.0 0.0; 0.0 -1.0]

@kwdef struct TFIM
    J::Float64 = 1.0
    h::Float64 = 0.50
end
function (model::TFIM)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1

        matising = copy(matarray_ising)
        matising[i] = σz
        matising[j] = σz
        matfield = copy(matarray_field)
        matfield[i] = σx
        H += model.J * reduce(⊗, matising)
        H += model.h * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
@kwdef struct TFIML
    J::Float64 = 1.0
    h::Float64 = 1.0
    g::Float64 = 0.10
end
function (model::TFIML)(N::Int)
    H = spzeros(2^N, 2^N)
    matarray = [I for _ in 1:N]
    matarray_ising = [I for _ in 1:N]
    matarray_field = [I for _ in 1:N]
    for i in 2:N
        j = i % N + 1

        matising = copy(matarray_ising)
        matising[i] = σz
        matising[j] = σz
        H += model.J * reduce(⊗, matising)

        matfield = copy(matarray_field)
        matfield[i] = σx
        H += model.h * reduce(⊗, matfield)
        matfield[i] = σz
        H += model.g * reduce(⊗, matfield)
    end
    return Hermitian(H)
end
model = TFIML()
H = model(20)
(λ, v) = eigen(H)

\delta


In [ ]:
using DiagonalizeHamiltonian
model = TFIM()
N = 4
basis = [↑, ↓]
# j site translation
function translate_state(N::Int, state_int::Int, j::Int)
    s = bitstring(state_int)[end-N+1:end]
    s_translated = s[N-j+1:N] * s[1:N-j]
    return parse(Int, s_translated; base=2)
    # return s_translated
end
# bitstring(6)[end-4+1:end]
# translate_state(4, 6, 1)

function get_representative_info(N::Int)
    # basis : length
    representative_map = Dict{Int,Int}()
    full_map = Dict{Int,Int}()
    for state_int in 0:(2^N-1)
        if haskey(full_map, state_int)
            # already processed
            continue
        end
        orbit = Int[state_int]
        current = state_int
        for j in 0:(N-1)
            current = translate_state(N, current, 1)
            if current == state_int
                break
            end
            push!(orbit, current)
        end

        L_n = length(orbit)
        rep_state = minimum(orbit)

        representative_map[rep_state] = L_n
        for s in orbit
            full_map[s] = rep_state
        end
    end
    return representative_map
end
# get_representative_info(4)
# get_representative_info(32)

function block_diagonalize_K(H::AbstractMatrix{T}, N::Int, K::Real) where T
    # K: 運動量 quantum number (例: 0.0, 2pi/N, 4pi/N, ...)

    rep_info = get_representative_info(N)
    rep_states = sort(collect(keys(rep_info)))
    D_K = length(rep_states)

    H_K = zeros(T, D_K, D_K)

    # 3. ブロック行列の要素 H_K[m, n] を計算
    for (m_idx, m_state) in enumerate(rep_states)
        L_m = rep_info[m_state]

        for (n_idx, n_state) in enumerate(rep_states)
            L_n = rep_info[n_state]

            # --- ブロック行列要素 H_K[m, n] の計算 ---
            # H_K[m, n] = 1/sqrt(L_m * L_n) * Σ_j exp(i K j) * <phi_m | H | T^j | phi_n>

            # 代表基底 |phi_m> の元のインデックス
            i_m = m_state + 1

            sum_of_elements = zero(T)

            # T^j |phi_n> を計算
            for j in 0:(L_n-1)
                # T^j |phi_n> の元のインデックス
                j_state = translate_state(N, n_state, j)
                i_j = j_state + 1

                # 行列要素 H[i_m, i_j] を取得
                H_element = H[i_m, i_j]

                # 運動量の位相因子を乗算
                # exp(-i K j) を使用 (H はエルミートなので、この定義でOK)
                sum_of_elements += exp(-im * K * j) * H_element
            end

            H_K[m_idx, n_idx] = sum_of_elements * sqrt(L_n) / N
        end
    end

    return Hermitian(H_K)
end


In [2]:
using DiagonalizeHamiltonian

# j site translation
function translate_state(N::Int, state_int::Int, j::Int)
    s = bitstring(state_int)[end-N+1:end]
    s_translated = s[N-j+1:N] * s[1:N-j]
    return parse(Int, s_translated; base=2)
end
function get_representative_info(N::Int)
    representative_map = Dict{Int,Int}()
    full_map = Dict{Int,Int}()
    for state_int in 0:(2^N-1)
        if haskey(full_map, state_int)
            continue
        end
        orbit = Int[state_int]
        current = state_int
        for j in 0:(N-1)
            current = translate_state(N, current, 1)
            if current == state_int
                break
            end
            push!(orbit, current)
        end
        L_n = length(orbit)
        rep_state = minimum(orbit)
        representative_map[rep_state] = L_n
        for s in orbit
            full_map[s] = rep_state
        end
    end
    return representative_map
end
function block_diagonalize_K(H::AbstractMatrix{T}, N::Int, K::Real) where T
    rep_info = get_representative_info(N)
    rep_states = sort(collect(keys(rep_info)))
    D_K = length(rep_states)
    H_K = zeros(T, D_K, D_K)
    for (m_idx, m_state) in enumerate(rep_states)
        L_m = rep_info[m_state]
        for (n_idx, n_state) in enumerate(rep_states)
            L_n = rep_info[n_state]
            # H_K[m, n] = 1/sqrt(L_m * L_n) * Σ_j exp(i K j) * <phi_m | H | T^j | phi_n>

            # 代表基底 |phi_m> の元のインデックス
            i_m = m_state + 1
            sum_of_elements = zero(T)
            # T^j |phi_n> を計算
            for j in 0:(L_n-1)
                # T^j |phi_n> の元のインデックス
                j_state = translate_state(N, n_state, j)
                i_j = j_state + 1
                # 行列要素 H[i_m, i_j] を取得
                H_element = H[i_m, i_j]
                # exp(-i K j) を使用 (H はエルミートなので、この定義でOK)
                sum_of_elements += exp(-im * K * j) * H_element
            end
            H_K[m_idx, n_idx] = sum_of_elements * sqrt(L_n) / N
        end
    end
    return Hermitian(H_K)
end

model = TFIM()
N = 4
K = 0.0
H = model(N)
K = 2.0
H_K = block_diagonalize_K(H, N, K)


6×6 Hermitian{ComplexF64, Matrix{ComplexF64}}:
      1.0+0.0im        …        0.0+0.0im         0.0+0.0im
 0.222595-0.0317301im           0.0+0.0im         0.0+0.0im
      0.0-0.0im            0.145963-0.227324im    0.0+0.0im
      0.0-0.0im           0.0865891+0.189201im    0.0+0.0im
      0.0-0.0im                 0.0+0.0im       0.125+0.0im
      0.0-0.0im        …      0.125-0.0im         1.0+0.0im